In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

tf.enable_eager_execution()

pd.set_option('float_format', '{:f}'.format)

In [5]:
tf.__version__

'1.13.1'

In [ ]:
!wget --header="Host: uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/74.0.3729.169 Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en,bg;q=0.9,es;q=0.8,de;q=0.7,fr;q=0.6,it;q=0.5,ja;q=0.4,pt;q=0.3,ru;q=0.2,sr;q=0.1,vi;q=0.1,el;q=0.1" --header="Referer: https://recsys.trivago.cloud/" "https://uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com/zip_by_token_key?dl=1&key=AilA9SlbHXj16VWMNJQyoa9w-L0TnjlKhCmezD8W46gwz0pBT2R7q6kP4Mja95PKD7aT2NdCO3B0Uy6dRK7EPi-sIJYYKZjP1jkRnlIrgyBwLdXJhdhrx3qc8QYBGIuDnnkFa6watmY4GEpnzh7eXBOdILeGVNdYo3mp9HODIjF1bHHfzMJIVJjX1h7ksBmyay8xyx5lE8utbI4NbgkR_M7tvJU8OPvOjxh3k7rmFJaIKcg3RnN5YpHwiKnKYvNzPAf1mglE14GHhsdCxwCrHkgCLZMJvhMnhvCyHjKVM7NW2onjo09ENsHttSoMy1-PC0U" -O "trivagoRecSysChallengeData2019_v2.zip" -c

In [6]:
!unzip trivago*

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
%%time
df_train = pd.read_csv('train.csv')
df_meta = pd.read_csv('item_metadata.csv')
df_test = pd.read_csv('test.csv')

Wall time: 56.9 s


In [12]:
df_meta.sample(3)

,item_id,properties
165738,5956454,WiFi (Public Areas)|Restaurant|Sitting Area (R...
480071,6006934,Satisfactory Rating|Very Good Rating|House / A...
64330,8879956,From 3 Stars|Good Rating|Telephone|Hotel|Air C...


In [13]:
df_train.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


### Reference valued

In [14]:
sfp = df_train[df_train['action_type'] == 'search for poi']['reference'].unique()
len(sfp), sfp[:3]

(13352, array(['Newtown', 'Surry Hills', 'Covent Garden'], dtype=object))

In [15]:
sfd = df_train[df_train['action_type'] == 'search for destination']['reference'].unique()
len(sfd), sfd[:3]

(22844, array(['London, United Kingdom', 'Bakersfield, USA',
        'Panama City, Panama'], dtype=object))

In [16]:
fs = df_train[df_train['action_type'] == 'filter selection']['reference'].unique()
len(fs), fs[:3]

(201, array(['Very Good Rating', 'Hostal (ES)', 'Hotel'], dtype=object))

In [17]:
coso = df_train[df_train['action_type'] == 'change of sort order']['reference'].unique()
len(coso), coso[:3]

(8, array(['interaction sort button', 'price and recommended', 'price only'],
       dtype=object))

### Meta valued

In [18]:
def get_meta_values(action_type):
    refs = df_train[df_train['action_type'] == action_type]['reference'].unique()
    ids = {int(i) for i in refs if i.isdigit()}
    props = df_meta[df_meta['item_id'].isin(ids)]['properties'].apply(lambda x: x.split('|'))
    return {i for items in props for i in items}

In [19]:
sfi = get_meta_values('search for item')
len(sfi), list(sfi)[:3]

(157, ['Childcare', 'Hydrotherapy', 'Computer with Internet'])

In [20]:
iid = get_meta_values('interaction item deals')
len(iid), list(iid)[:3]

(157, ['Childcare', 'Hydrotherapy', 'Electric Kettle'])

In [22]:
iii = get_meta_values('interaction item image')
len(iii), list(iii)[:3]

(157, ['Childcare', 'Hydrotherapy', 'Electric Kettle'])

In [21]:
iiinf = get_meta_values('interaction item info')
len(iiinf), list(iiinf)[:3]

(157, ['Childcare', 'Hydrotherapy', 'Computer with Internet'])

In [23]:
iir = get_meta_values('interaction item rating')
len(iir), list(iir)[:3]

(157, ['Childcare', 'Hydrotherapy', 'Computer with Internet'])

### Clickout item

In [24]:
df_train[df_train['action_type'] == 'clickout item'].head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...


---

In [26]:
df_train.groupby('session_id')['step'].count().describe()

count   910683.000000
mean        17.495651
std         48.181687
min          1.000000
25%          2.000000
50%          4.000000
75%         13.000000
max       3522.000000
Name: step, dtype: float64

In [245]:
def extract_seqs(group):
    group['session_id']
    group['action_type']

In [227]:
groups = df_train.groupby('session_id')

In [232]:
df = df_train.sample(frac=0.01)

In [233]:
len(df)

159330

In [241]:
groups = df.groupby('session_id')

In [250]:
%%time
train = pd.DataFrame({
    'action_type_seq': groups.apply(lambda x: x['action_type'].tolist()),
    'reference_seq': groups.apply(lambda x: x['reference'].tolist()),
}).reset_index()

CPU times: user 8.68 s, sys: 3.24 ms, total: 8.69 s
Wall time: 8.69 s


In [254]:
train[:10]

,session_id,action_type_seq,reference_seq
0,00000510f1adc,[clickout item],[7281198]
1,000066611146f,[clickout item],[5479306]
2,00019f78d2c36,[interaction item image],[4755476]
3,0001e610ce498,"[interaction item image, interaction item image]","[1257216, 488901]"
4,0001fe0007954,[change of sort order],[price and recommended]
5,00030305c6aa7,[filter selection],[Hostal (ES)]
6,00033ccbbcdd4,[interaction item image],[3854580]
7,00040aa1d0a4d,[interaction item image],[104874]
8,0005174c23dcb,[interaction item info],[1134551]
9,000543ecdcf0f,"[interaction item image, interaction item imag...","[3758872, 3758872, 124561, 3758872]"


In [263]:
impr = groups.apply(lambda x: x[~x['impressions'].isna()]['impressions'].tolist())

---

In [33]:
dataset = tf.data.Dataset.from_tensor_slices(
   {"a": tf.random_uniform([4]),
    "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})

In [35]:
sess = tf.Session()

In [36]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
  value = sess.run(next_element)
  assert i == value

In [7]:
dataset = tf.data.experimental.CsvDataset(
    ['item_metadata.csv'],
    [tf.int32, tf.string],
    header=True
)

dataset = dataset.map(lambda idx, row: (idx, tf.string_split([row], '|')))

In [8]:
next_item = dataset.make_one_shot_iterator().get_next()

Instructions for updating:
Colocations handled automatically by placer.
